In [1]:
from openai import OpenAI
import yaml
import os
import numpy as np
import pandas as pd

from gpt.curriculum_api import CurriculumAPI
from gpt.utils import *

In [2]:
prompt_path = "./gpt/prompt/"
env_name = "Fetch_Push"

API = CurriculumAPI(env_name, prompt_path)

In [3]:
tasks_details = API.generate_curriculum()

Task 1
Name: Basic Movement
Description: maximize torso_velocity
Reason: This task aims to teach the ant basic movement skills. Increasing torso velocity will ensure it can move from its initial position, which is crucial for later tasks requiring navigation towards a goal.

Task 2
Name: Orientation Maintenance
Description: maintain torso_orientation as a value of [1.0, 0.0, 0.0, 0.0]
Reason: Maintaining a stable orientation is crucial for effective movement and navigation through the maze. This task ensures the ant learns how to stabilize itself, which is essential for performing precise movements towards the goal.

Task 3
Name: Angular Velocity Control
Description: minimize torso_angular_velocity
Reason: This task teaches the ant to control and minimize its angular velocity. A lower angular velocity is important for precision and stability, especially in a maze environment where sudden changes in direction can lead to failure.

Task 4
Name: Navigation to Intermediate Goals
Descriptio

In [4]:
env_code_path = "./environments/Curriculum/envs/Fetch_Push.py"
task = tasks_details[0]

code_1 = """
def compute_reward_curriculum(self):
    ant_obs = self.get_ant_obs()
    goal_distance = self.goal_distance(ant_obs)
    velocity_weight = 0.1
    goal_distance_weight = 1.0

    # Reward for moving towards the goal
    reward_for_goal_distance = -np.tanh(goal_distance) * goal_distance_weight

    # Reward for maintaining a certain velocity (encourage movement)
    torso_velocity = self.torso_velocity(ant_obs)
    desired_velocity = np.array([1.0, 0.0]) # Example desired velocity towards the goal
    velocity_difference = np.linalg.norm(torso_velocity - desired_velocity)
    reward_for_velocity = -np.tanh(velocity_difference) * velocity_weight

    # Total reward
    total_reward = reward_for_goal_distance + reward_for_velocity

    reward_dict = {
        'goal_distance': reward_for_goal_distance,
        'velocity': reward_for_velocity
    }

    return total_reward, reward_dict
"""
previous_reward_code = [code_1]

reward_code = API.update_env_code(env_code_path, task, previous_reward_code)

"```python
def compute_reward_curriculum(self):
    ant_obs = self.get_ant_obs()
    torso_vel = self.torso_velocity(ant_obs)
    
    # Reward components
    velocity_reward_weight = 1.0
    
    # Reward calculations
    velocity_reward = np.tanh(np.linalg.norm(torso_vel)) # Maximize torso velocity
    
    # Total reward calculation
    total_reward = velocity_reward_weight * velocity_reward
    
    # Reward dictionary
    reward_dict = {
        'velocity_reward': velocity_reward
    }
    
    return total_reward, reward_dict
```"
Extracted Code Block:
 def compute_reward_curriculum(self):
    ant_obs = self.get_ant_obs()
    torso_vel = self.torso_velocity(ant_obs)
    
    # Reward components
    velocity_reward_weight = 1.0
    
    # Reward calculations
    velocity_reward = np.tanh(np.linalg.norm(torso_vel)) # Maximize torso velocity
    
    # Total reward calculation
    total_reward = velocity_reward_weight * velocity_reward
    
    # Reward dictionary
    reward_dict = 

In [5]:
import Curriculum
from utils.train_utils import *
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env

env_id = "Curriculum/Fetch_Push-v0"
num_cpu = 4
env = make_vec_env("Curriculum/Fetch_Push-v0")

2024-03-25 19:14:08.606370: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-25 19:14:08.606393: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-25 19:14:08.606406: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-25 19:14:08.610315: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/kh-ryu/anaconda3/envs/curriculum/lib/python3

In [6]:
obs = env.reset()
for i in range(100):
    action = [env.action_space.sample(), env.action_space.sample(), env.action_space.sample(), env.action_space.sample()]
    obs, rewards, dones, info = env.step(action)
    print(rewards)

[-0.8548721]
[-0.8569425]
[-0.90360034]
[-0.67222303]
[-0.7980592]
[-0.48293692]
[-0.37922052]
[-0.32729062]
[-0.8477994]
[-0.52121484]
[-0.27145526]
[-0.49351692]
[-0.43549073]
[-0.4660426]
[-0.33359784]
[-0.2809185]
[-0.22043951]
[-0.18211484]
[-0.34708044]
[-0.18931317]
[-0.28505123]
[-0.16576925]
[-0.27528203]
[-0.34637013]
[-0.49647218]
[-0.72735095]
[-0.4839832]
[-0.4058162]
[-0.45727134]
[-0.45051712]
[-0.69855565]
[-0.91694945]
[-0.3660304]
[-0.4896844]
[-0.33393237]
[-0.7600975]
[-0.43458778]
[-0.41114312]
[-0.49821347]
[-0.3092446]
[-0.16893642]
[-0.16343866]
[-0.18539545]
[-0.21579468]
[-0.42603213]
[-0.54612494]
[-0.4008043]
[-0.38511106]
[-0.6391636]
[-0.67948455]
[-0.49913564]
[-0.87358236]
[-0.3018769]
[-0.5179596]
[-0.4549645]
[-0.26950166]
[-0.42400232]
[-0.15804742]
[-0.16362563]
[-0.27348998]
[-0.17178498]
[-0.15780343]
[-0.16048759]
[-0.18210576]
[-0.20815045]
[-0.1710233]
[-0.1437928]
[-0.13402286]
[-0.3011705]
[-0.17997427]
[-0.1530135]
[-0.18660095]
[-0.13074161]